In [ ]:
import sys
import os
import psycopg2
import numpy as np
import random
import json
import boto3

# Connection details
DB_HOST = "prod-postgres-db.c3ik622s2eej.eu-west-1.rds.amazonaws.com"
DB_NAME = "postgres"
DB_USER = "db_admin"
DB_PASS = "^bdnWKXhljqLl37B"
DB_PORT = "5432"^bdnWKXhljqLl37B

In [5]:
def get_db_connection():
    try:
        conn = psycopg2.connect(
            host=DB_HOST,
            database=DB_NAME,
            user=DB_USER,
            password=DB_PASS,
            port=DB_PORT
        )
        conn.autocommit = True
        return conn
    except Exception as e:
        print(f"Connection failed: {e}")
        return None

def reset_and_optimize_table():
    conn = get_db_connection()
    if not conn: return

    cur = conn.cursor()
    
    try:
        print("1. Enabling pgvector extension...")
        cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
        
        print("2. DROPPING existing table 'embeddings' (Clean Slate)...")
        cur.execute("DROP TABLE IF EXISTS embeddings;")
        
        print("3. Creating NEW Optimized Table with Hybrid Search support...")
        # We add 'fts' (Full Text Search) column
        create_table_query = """
        CREATE TABLE embeddings (
            id BIGSERIAL PRIMARY KEY,
            document_id TEXT NOT NULL,
            chunk_id INTEGER NOT NULL,
            content TEXT,
            embedding vector(1024), 
            fts tsvector,  -- <--- NEW: For Keyword Search
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        """
        cur.execute(create_table_query)
        
        print("4. Creating Trigger for Automatic Keyword Tokenization...")
        # This function ensures that whenever you insert 'content', 
        # the 'fts' column is automatically populated with search tokens.
        trigger_func = """
        CREATE OR REPLACE FUNCTION embeddings_tsvector_trigger() RETURNS trigger AS $$
        BEGIN
          NEW.fts := to_tsvector('english', NEW.content);
          RETURN NEW;
        END
        $$ LANGUAGE plpgsql;
        """
        cur.execute(trigger_func)
        
        trigger_create = """
        CREATE TRIGGER tsvectorupdate BEFORE INSERT OR UPDATE
        ON embeddings FOR EACH ROW EXECUTE FUNCTION embeddings_tsvector_trigger();
        """
        cur.execute(trigger_create)

        print("5. Creating Indexes (HNSW for Vector, GIN for Text)...")
        # HNSW for fast vector search (Cosine Similarity)
        cur.execute("CREATE INDEX idx_embeddings_vec ON embeddings USING hnsw (embedding vector_cosine_ops);")
        # GIN for fast keyword search
        cur.execute("CREATE INDEX idx_embeddings_fts ON embeddings USING GIN (fts);")
        
        print("SUCCESS: Database is now optimized for Hybrid Search!")
        
    except Exception as e:
        print(f"Error setting up database: {e}")
    finally:
        cur.close()
        conn.close()

# RUN THE RESET
reset_and_optimize_table()

1. Enabling pgvector extension...
2. DROPPING existing table 'embeddings' (Clean Slate)...
3. Creating NEW Optimized Table with Hybrid Search support...
4. Creating Trigger for Automatic Keyword Tokenization...
5. Creating Indexes (HNSW for Vector, GIN for Text)...
SUCCESS: Database is now optimized for Hybrid Search!


In [3]:
import random

def verify_insertion_logic():
    conn = get_db_connection()
    if not conn: return

    cur = conn.cursor()
    try:
        print("Testing Hybrid Insertion...")
        
        # Dummy data: Note we do NOT pass 'fts', Postgres handles it
        doc_id = "test-hybrid-doc"
        content_text = "Amazon Bedrock is a fully managed service that makes foundation models available through an API."
        dummy_embedding = [random.uniform(-1, 1) for _ in range(1024)]
        
        cur.execute(
            """
            INSERT INTO embeddings (document_id, chunk_id, content, embedding) 
            VALUES (%s, %s, %s, %s::vector) 
            RETURNING id, fts;
            """,
            (doc_id, 0, content_text, dummy_embedding)
        )
        
        row = cur.fetchone()
        print(f"Inserted Row ID: {row[0]}")
        print(f"Generated TSVector: {row[1]}") # Should look like 'amazon':1 'api':14 ...
        
        if row[1]:
            print("VERIFICATION SUCCESS: Text was automatically tokenized for keyword search.")
        else:
            print("VERIFICATION FAILED: fts column is empty.")
            
    except Exception as e:
        print(f"Insertion test failed: {e}")
    finally:
        # Cleanup
        cur.execute("DELETE FROM embeddings WHERE document_id = 'test-hybrid-doc'")
        cur.close()
        conn.close()

verify_insertion_logic()

Testing Hybrid Insertion...
Inserted Row ID: 1
Generated TSVector: 'amazon':1 'api':15 'avail':12 'bedrock':2 'foundat':10 'fulli':5 'make':9 'manag':6 'model':11 'servic':7
VERIFICATION SUCCESS: Text was automatically tokenized for keyword search.


In [3]:
bedrock_client = boto3.client('bedrock-runtime', region_name='eu-west-1')

def get_titan_embedding(text):
    body = json.dumps({"inputText": text})
    response = bedrock_client.invoke_model(
        body=body,
        modelId='amazon.titan-embed-text-v2:0',
        contentType='application/json',
        accept='application/json'
    )
    response_body = json.loads(response.get('body').read())
    return response_body.get('embedding')

def perform_hybrid_search(user_query):
    print(f"--- Hybrid Search (OpenSearch Style) ---")
    print(f"Query: '{user_query}'")
    
    conn = get_db_connection()
    if not conn: return

    try:
        # 1. Generate Embedding
        query_vec = get_titan_embedding(user_query)
        
        # 2. Prepare Keyword Query (Simple cleaning for websearch syntax)
        # websearch_to_tsquery handles quotes and simple logic nicely
        keyword_q = user_query.replace("'", "") 
        
        cur = conn.cursor()
        
        # 3. The RRF (Reciprocal Rank Fusion) Query
        # This combines Vector Rank + Keyword Rank
        sql = """
        WITH semantic_search AS (
            SELECT id, content, RANK() OVER (ORDER BY embedding <=> %s::vector) as rank
            FROM embeddings
            ORDER BY embedding <=> %s::vector
            LIMIT 20
        ),
        keyword_search AS (
            SELECT id, content, RANK() OVER (ORDER BY ts_rank_cd(fts, websearch_to_tsquery('english', %s)) DESC) as rank
            FROM embeddings
            WHERE fts @@ websearch_to_tsquery('english', %s)
            LIMIT 20
        )
        SELECT 
            COALESCE(s.content, k.content) as content,
            (COALESCE(1.0 / (60 + s.rank), 0.0) + COALESCE(1.0 / (60 + k.rank), 0.0)) as score
        FROM semantic_search s
        FULL OUTER JOIN keyword_search k ON s.id = k.id
        ORDER BY score DESC
        LIMIT 5;
        """
        
        cur.execute(sql, (query_vec, query_vec, keyword_q, keyword_q))
        results = cur.fetchall()
        
        if not results:
            print("No results found. (Did you re-index your documents after running Cell 1?)")
            return

        print(f"\nTop 5 Hybrid Matches:\n")
        for i, (content, score) in enumerate(results):
            print(f"[{i+1}] Score: {score:.5f}")
            print(f"Content: {content[:200]}...")
            print("-" * 40)
            
    except Exception as e:
        print(f"Hybrid Search Error: {e}")
    finally:
        conn.close()

# Example Usage
# perform_hybrid_search("What is Amazon Bedrock AgentCore?")

In [ ]:
# Example Usage
# perform_hybrid_search("What commissions does Artbridge charge?")